In [ ]:
%matplotlib inline

# Non-GP regressor.


In [ ]:
from __future__ import annotations

from gemseo import configuration
from gemseo import sample_disciplines
from gemseo.mlearning.regression.algos.rbf import RBFRegressor

from gemseo_mlearning.active_learning.active_learning_algo import ActiveLearningAlgo
from gemseo_mlearning.active_learning.distributions.regressor_distribution import (
    RegressorDistribution,
)
from gemseo_mlearning.problems.rosenbrock.rosenbrock_discipline import (
    RosenbrockDiscipline,
)
from gemseo_mlearning.problems.rosenbrock.rosenbrock_space import RosenbrockSpace

# Update the configuration of |g| to speed up the script.
configuration.fast = True

The use of active learning methods
dedicated to optimization
is illustrated in this example,
with all default settings.
The function to minimize is
the Rosenbrock function $f(x_1,x_2)=(1-x_1)^2+100(x_2-x_1^2)^2$:



In [ ]:
discipline = RosenbrockDiscipline()

with $x_1$ and $x_2$ belonging to $[-2,2]^2$:



In [ ]:
input_space = RosenbrockSpace()

First,
we create an initial training dataset using an optimal LHS including 10 samples:



In [ ]:
learning_dataset = sample_disciplines(
    [discipline], input_space, "y", algo_name="OT_OPT_LHS", n_samples=10
)

and a universal regressor, namely a radial basis function network based on SciPy:



In [ ]:
regressor = RBFRegressor(learning_dataset)
regressor_distribution = RegressorDistribution(regressor, bootstrap=False)
regressor_distribution.learn()

Then,
we look for 20 points that will help us
to find the minimum.
By default,
for this purpose,
the active learning algorithm looks
for the point maximizing the expected improvement
with the help of the SLSQP gradient-based algorithm
applied in a multistart framework.



In [ ]:
active_learning = ActiveLearningAlgo("Minimum", input_space, regressor_distribution)
active_learning.acquire_new_points(discipline, 20)

Then,
we plot the history of the quantity of interest



In [ ]:
active_learning.plot_qoi_history()

as well as
the training points,
the original model,
the RBF regressor
and the expected improvement
after the last acquisition:



In [ ]:
active_learning.plot_acquisition_view(discipline=discipline)

Finally,
we compare the estimated minimum
from the active learning procedure
to its exact theoretical value:

